<a href="https://colab.research.google.com/github/jwkanggist/EverybodyTensorflow2.0/blob/master/lab17_resnet34_cifar100_tf2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#LAB17: ResNet34 for Cifar100

Skip connection을 이용하는 ResNet34을 만들어 보자
- Cifar100 datatset을 위한 실험


In [1]:
# preprocessor parts

from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.callbacks import TensorBoard

import numpy as np
import pandas as pd

from matplotlib import cm
from matplotlib import gridspec
import matplotlib.pyplot as plt

from datetime import datetime

TensorFlow 2.x selected.


In [2]:
  # for Tensorboard use
  LOG_DIR = 'drive/data/tb_logs'
	
  !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
  !unzip ngrok-stable-linux-amd64.zip
	
  import os
  if not os.path.exists(LOG_DIR):
    os.makedirs(LOG_DIR)
	  
  get_ipython().system_raw(
      'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
      .format(LOG_DIR))
	
  get_ipython().system_raw('./ngrok http 6006 &')
	
  !curl -s http://localhost:4040/api/tunnels | python3 -c \
      "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

--2020-01-17 17:19:57--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 52.203.255.14, 52.203.65.11, 34.231.143.21, ...
Connecting to bin.equinox.io (bin.equinox.io)|52.203.255.14|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  6.47MB/s    in 2.0s    

2020-01-17 17:20:00 (6.47 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   
https://be92353d.ngrok.io


In [3]:
# dataset loading part
# 데이터 파이프라인 부분 

cifar100 = tf.keras.datasets.cifar100

(x_train, y_train), (x_test, y_test) = cifar100.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

print("x_train.shape = %s" % str(x_train.shape))
print("y_train.shape = %s" % str(x_train.shape))

x_train = x_train.reshape([x_train.shape[0],
                           x_train.shape[1],
                           x_train.shape[2],3])

x_test = x_test.reshape([x_test.shape[0],
                           x_test.shape[1],
                           x_test.shape[2],3])


x_train = np.array(tf.keras.backend.resize_images(x_train, height_factor=2,width_factor=2, data_format='channels_last',interpolation='bilinear'))
x_test = np.array(tf.keras.backend.resize_images(x_test, height_factor=2,width_factor=2, data_format='channels_last',interpolation='bilinear'))

print("x_train.shape = %s" % str(x_train.shape))
print("x_test.shape = %s" % str(x_test.shape))

print("y_train.shape = %s" % str(y_train.shape))
print("y_test.shape = %s" % str(y_test.shape))

input_shape = x_train.shape[1]

169009152/169001437 [==============================] - 11s 0us/step
x_train.shape = (50000, 32, 32, 3)
y_train.shape = (50000, 32, 32, 3)
x_train.shape = (50000, 64, 64, 3)
x_test.shape = (10000, 64, 64, 3)
y_train.shape = (50000, 1)
y_test.shape = (10000, 1)


In [0]:
# Network modules
def residual_layer_type1(net_in,
                   filters1,kernel_size1,strides1,
                   filters2,kernel_size2,strides2):
  
  net = tf.keras.layers.BatchNormalization()(net_in)
  net = tf.keras.layers.Activation(activation='relu')(net)
  net = tf.keras.layers.Conv2D(filters1,kernel_size=(kernel_size1,kernel_size1),
                               strides=(strides1,strides1),
                               padding='same',
                               kernel_initializer='he_normal',
                               activation=None)(net)

  net = tf.keras.layers.BatchNormalization()(net)
  net = tf.keras.layers.Activation(activation='relu')(net)
  net = tf.keras.layers.Conv2D(filters2,kernel_size=(kernel_size2,kernel_size2),
                               strides=(strides2,strides2),
                               padding='same',
                               kernel_initializer='he_normal',
                               activation=None)(net)

  shortcut = tf.keras.layers.Conv2D(filters2,kernel_size=(1,1),
                               strides=(strides1,strides1),
                               padding='same',
                               kernel_initializer='he_normal',
                               activation=None)(net_in)
  net = tf.keras.layers.add([net,shortcut])

  return net


def residual_layer_type2(net_in,
                   filters1,kernel_size1,strides1,
                   filters2,kernel_size2,strides2):
  
  net = tf.keras.layers.BatchNormalization()(net_in)
  net = tf.keras.layers.Activation(activation='relu')(net)
  net = tf.keras.layers.Conv2D(filters1,kernel_size=(kernel_size1,kernel_size1),
                               strides=(strides1,strides1),
                               padding='same',
                               kernel_initializer='he_normal',
                               activation=None)(net)

  net = tf.keras.layers.BatchNormalization()(net)
  net = tf.keras.layers.Activation(activation='relu')(net)
  net = tf.keras.layers.Conv2D(filters2,kernel_size=(kernel_size2,kernel_size2),
                               strides=(strides2,strides2),
                               padding='same',
                               kernel_initializer='he_normal',
                               activation=None)(net)


  net = tf.keras.layers.add([net,net_in])

  return net

# moblienet model
net_in = tf.keras.layers.Input(shape=(input_shape,input_shape,3))
net = tf.keras.layers.Conv2D(64,kernel_size=(9,9),strides=1,padding='valid', kernel_initializer='he_normal',activation='relu')(net_in)
net = residual_layer_type2(net,64,3,1,64,3,1)
net = residual_layer_type2(net,64,3,1,64,3,1)
net = residual_layer_type2(net,64,3,1,64,3,1)
net = residual_layer_type1(net,128,3,2,128,3,1)
net = residual_layer_type2(net,128,3,1,128,3,1)
net = residual_layer_type2(net,128,3,1,128,3,1)
net = residual_layer_type2(net,128,3,1,128,3,1)
net = residual_layer_type1(net,256,3,2,256,3,1)
net = residual_layer_type2(net,256,3,1,256,3,1)
net = residual_layer_type2(net,256,3,1,256,3,1)
net = residual_layer_type2(net,256,3,1,256,3,1)
net = residual_layer_type2(net,256,3,1,256,3,1)
net = residual_layer_type2(net,256,3,1,256,3,1)
net = residual_layer_type1(net,512,3,2,512,3,1)
net = residual_layer_type2(net,512,3,1,512,3,1)
net = residual_layer_type2(net,512,3,1,512,3,1)
net = tf.keras.layers.GlobalAvgPool2D(data_format='channels_last')(net)
net = tf.keras.layers.Dropout(0.4)(net)
net = tf.keras.layers.Dense(units=1000,activation='softmax')(net)
model = tf.keras.models.Model(inputs=net_in,outputs=net)


opt_fn = tf.keras.optimizers.Adam(learning_rate=1e-3,
                                beta_1=0.9,
                                beta_2=0.999)

# 'sparse_categorical_crossentropy' is for integer labels
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

tensorboard_callback = TensorBoard(log_dir=LOG_DIR, 
                           histogram_freq=1,
                           write_graph=True,
                           write_images=True)

In [8]:
# model training and evaluation part
training_epochs = 20
batch_size = 128
model.summary()
model.fit(x_train, y_train, 
          epochs=training_epochs,
          validation_data=(x_test, y_test),
          batch_size=batch_size,
          callbacks=[tensorboard_callback])
model.evaluate(x_test,  y_test, verbose=2)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv2d_20 (Conv2D)              (None, 56, 56, 64)   15616       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_16 (BatchNo (None, 56, 56, 64)   256         conv2d_20[0][0]                  
__________________________________________________________________________________________________
activation_16 (Activation)      (None, 56, 56, 64)   0           batch_normalization_16[0][0]     
______________________________________________________________________________________________

[3.180428157424927, 0.4934]

In [0]:
# prediction 

test_input = x_test[300,:,:,:]
pred_y = model.predict(test_input.reshape([1,64,64,3]))

plt.figure(1)
plt.imshow(test_input.reshape([64,64,3]))
plt.title("input")
print("model prediction = %s"% pred_y.argmax())
